In [ ]:
import glob
import numpy as np
import pandas as pd
import re

pattern = re.compile(r'(\d{4}\-\d{2})\-([-a-z]*)\-street\.csv$')
dataFiles = glob.glob('../data/**/*.csv', recursive=True)
dates, forces = set(), set()
dataColumns =  {}

In [ ]:
# Function to calculate month offset

def month_Offset(start, target):
    startYear, startMonth   = [int(x) for x in start.split('-')]
    targetYear, targetMonth = [int(x) for x in target.split('-')]
    
    return (targetYear - startYear) * 12 + targetMonth - startMonth

In [ ]:
def incrementDateByMonth(yearMonthStr):
    year, month = [int(x) for x in yearMonthStr.split('-')]
    
    if year < 1000:
        raise ValueError('Year out of bounds')
    month += 1
    if month > 12:
        month = 1
        year += 1
    return f'{year}-{month}'
        

In [ ]:
# prepare to accept data
for name in dataFiles:
    results = re.findall(pattern, name)
    date, force = results[0][0], results[0][1]
    dates.add(date)
    forces.add(force)

startDate = min(dates)
forces = sorted(forces)

for force in forces:
    dataColumns[force] = [np.NaN] * len(dates)

In [ ]:
# load in data from all of the files

for file in dataFiles:
    df = pd.read_csv(file, usecols=['Crime type'])
    
    results = re.findall(pattern, file)
    date, force = results[0][0], results[0][1]
    bikeThefts = df.groupby(['Crime type']).size()['Bicycle theft']

    dataColumns[force][month_Offset(startDate, date)] = bikeThefts

Using the 'usecols' option in read_csv cut down processing time of csv files by half, it would be good to use Dask to parallelise the processing.

In [ ]:
# Create DataFrame

bikeTheftsFrame = pd.DataFrame.from_dict(dataColumns)
endDate = incrementDateByMonth(max(dates))
bikeTheftsFrame.index = pd.date_range(startDate, endDate, freq='M')

In [ ]:
bikeTheftsFrame

The dataframe of the bike theft data for each force for each month, this allows any issues to be found before totalling up the values.

In [ ]:
bikeTheftsFrame.info()

In [ ]:
from matplotlib.colors import LogNorm
import seaborn as sns
%matplotlib inline

sns.set(rc={'figure.figsize':(20,18)})
logNorm = LogNorm(vmin=bikeTheftsFrame.min().min(), vmax=bikeTheftsFrame.max().max())

sns.heatmap(bikeTheftsFrame, norm=logNorm)

This shows the the log of bike theft data for each of the forces for each month, the logs allow one better insight into how the data varies.

There are four grey spots for the three forces which show NaN, since I set the default value to NaN so it would be more apparent.

In [ ]:
from sklearn.impute import SimpleImputer

fill_NaN = SimpleImputer(missing_values=np.nan, strategy='mean')
bikeTheftsFrame_clean = pd.DataFrame(fill_NaN.fit_transform(bikeTheftsFrame))
bikeTheftsFrame_clean.columns = bikeTheftsFrame.columns
bikeTheftsFrame_clean.index = bikeTheftsFrame.index
bikeTheftsFrame_clean = bikeTheftsFrame_clean.astype(int)

bikeTheftsFrame_clean.loc[:,'Total'] = bikeTheftsFrame_clean.sum(axis=1)

In [ ]:

sns.set(rc={'figure.figsize':(20,18)})
logNorm = LogNorm(vmin=bikeTheftsFrame.min().min(), vmax=bikeTheftsFrame.max().max())

sns.heatmap(bikeTheftsFrame_clean, norm=logNorm)

The NaNs have been removed with by using a column average, this is better than zero, and better than removing the three columns, however it is artificial data very roughly inferred from the surrounding dataset, and with no regard to the seasonal context of the missing datapoints.

**Note however that the number of bike thefts appear to dip over winter each year.**

In [ ]:
bikeTheftsFrame_clean.info()

In [ ]:
insuranceFrame = pd.DataFrame(bikeTheftsFrame_clean.Total)

Based on the provided dataset. Generate a month on month simplified bicycle insurance price graph.

Base the prices on the number of bicylce related crimes that happen each month.
Each crime accounts for a £1 increase in the price.

The price is cumulative so, if last month's price is £3, and there are two incidents per month,
the price would go up £2 per as follows: £3, £5, £7

For any month when the number of incidents is half of less than the number of incidents in the previous month,
the price goes down £1


*I am assuming the starting price is the previous month's 'Total'?*
*The price decrease of £1 does seem very small considering the how high the insurance price has risen to*

In [ ]:
def CalculateInsurancePrice(df):
    price = df.Total[0]
    df['InsurancePrice'] = 0
    
    for index in range(1, len(df)):
        thisMonth = df.Total[index]
        lastMonth = df.Total[index - 1]

        if thisMonth > lastMonth:
            price += thisMonth
        elif thisMonth <= lastMonth/2:
            price -= 1
        
        df.InsurancePrice[index] = price
        
CalculateInsurancePrice(insuranceFrame)
     

In [ ]:
insuranceFrame

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.rcParams['figure.figsize'] = (10,8)
plt.plot(insuranceFrame.InsurancePrice)
plt.title('Bicycle theft insurance')
plt.xlabel('Date')
plt.xticks(rotation=70)
plt.ylabel('Price of bicycle insurance (£)')

#plt.show()
plt.savefig('../figures/BicycleInsurancePrice.png')